# RAMSES Fundamentals

## 1. Prerequisites

### compilers

You will need a **fortran compiler** to compile RAMSES, along with some version of **MPI** if you have access to multiple cores and want to run RAMSES in parallel.

We recommend installing recent versions of `gcc` (incl. `gfortran`) and `openmpi`. 


### python

The tutorials require running `python3` in a `Jupyter notebook`. 

We will use a the following python packages: 
```
numpy, astropy, matplotlib
```

Depending on the tutorials, different analysis tools will also be used, including : 
```
yt, yt_astro_analysis, colossus, osiris, ... 
```

## 2. Getting RAMSES

The RAMSES code is available on github.com and the best practice is to get the code by cloning the repository on your computer with one of the following commands. If you did not set up an ssh key on github, use:
```
git clone https://github.com/ramses-organisation/ramses.git
```
if you do have an ssh key set up (recommended) you can use the following command instead:
```
git clone git@github.com:ramses-organisation/ramses.git
```

This will create a `ramses` directory containing the code. By default git gives you the stable branch, which is the one you should use for the tutorials. In order to check which branch of the code you are using, you may type: 
```
cd ramses
git branch
``` 
This should return:
```
* stable
```

Should you just want a copy of the code, without using git (this is not recommended), you may also download that directly from the github website https://github.com/ramses-organisation/ramses (click on the "<> Code" green button, and then "Download ZIP").

## 3. Compiling RAMSES




```
cd ramses/bin
make clean
make NDIM=3 MPI=1 PATCH=../patch/init/dice/ EXEC=ramses-dice
```